In [21]:
import imp
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
g = imp.load_source('gsync', 'modules/gCanvas.py')
c = imp.load_source('conf', 'config/conf.py')
r = imp.load_source('report', 'modules/reader.py')
s = imp.load_source('patch', 'modules/patcher.py')

Название аккаунта, который вносит изменения в Google Документ: *perek-test@outofcloud-analytics.iam.gserviceaccount.com*<br>
Json ключ лежит в папке "config/creds_gs"

In [24]:
# Устанавливаем связь с google документом, который будем изменять
# https://docs.google.com/spreadsheets/d/1N8yC457qlPtYhAdKkTNdJ0dv50iGIttTFzICh6tTGV4/edit#gid=1772378012
conn = g.Connection("new test dashboard")

Важный момент, скрипт ориентируется на дату заполнения, указанную в 3 строчке документа <br>
Это значит, что до запуска та неделя, которую Вы собираетесь обновлять - должна иметь пустое поле

### 2 вкладка

Обновление информации по покупкам строится на основе файла общего файла заказов для клиента, его необходимо обновлять при запуске с помощью еженедльной выгрузки заказов клиента с помощью команды ниже.

In [25]:
r.append_new_orders("data/cohort/orders.f", 'orders_2020-02-18_10-51-49.csv')

,date_ord,id_cust,id_ord,revenue
0,2019-04-01,316333,81234763,2104
1,2019-04-01,426779,81255520,2609
2,2019-04-01,428351,81255523,3516
3,2019-04-01,473075,81255526,9580
4,2019-04-01,557462,81255535,3297
...,...,...,...,...
1095903,2020-02-17,931345,83747231,2571
1095904,2020-02-17,1032649,81975569,2543
1095905,2020-02-17,922413,83752999,2972
1095906,2020-02-17,411215,83747214,3729


Обновление информации по покупкам строится на основе файла общего файла заказов для клиента, его необходимо обновлять при запуске с помощью еженедльной выгрузки заказов клиента с помощью команды ниже.

In [ ]:
ws = conn.get_sheet_by_name("2. Триггеры — инкр. нед.")
orders_df = pd.read_feather('data/cohort/orders.f')
orders = r.get_order_info(orders_df, 'group')
patch = s.Patcher(orders, 1, ws.max_cols)
sh.update_batch(patch.patch)

### 3 вкладка

3. Триггеры — стат. нед. - отчет по автоматическим рассылкам строится на основе файла отчета из MindBox,
с помощью выгрузки: Отчёты -> Сводная статистика по рассылкам -> Выбор дат -> Экспорт CSV файла

In [3]:
# Подготовка недельного MindBox отчета
df = r.get_report('report.xlsx', 0)
df = df[df.channel != 'Ручные рассылки']

In [ ]:
#Подключение к Google API
ws = conn.get_sheet_by_name("3. Триггеры — стат. нед.")
gdf = g.get_as_dataframe(ws, index_col = 0, evaluate_formulas=True)
ws = g.gCanvas(ws)

In [8]:
patcher = s.Patcher(df, 2, ws.max_cols, gdf)

Определите категорию кампании для 9 новых:

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер 17. Брошенная корзина с оффером: 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Брошенная корзина с оффером
Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> NY landing 2020 DOI promocode: 
2
Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets

Нажмите 'Enter' для Email
	'1' для Web-push
	'2' для сезонной
	'3' для SMS

-> Триггер_14_ "Онлайн дешевле": 

Нажмите только 'Enter', если название компании Вас устраивает:
Введите новое название для отображения в Google Sheets
Онлай

In [ ]:
#Обновление документа
ws.update_with_df(patch.gdf)
ws.format(patch.format)

### 4 вкладка

4. Триггеры — инкр. месяц — стат. нед. - отчет по автоматическим рассылкам строится на основе файла отчета из MindBox, с помощью выгрузки: Отчёты -> Сводная статистика по рассылкам -> Выбор дат -> Экспорт CSV файла

In [ ]:
# MindBox report
df = r.get_report('report.xlsx', 0)
df = df[df.channel == 'Ручные рассылки']

In [ ]:
ws = conn.get_sheet_by_name("4. Триггеры — инкр. месяц")
ws = g.gCanvas(ws)
patch = s.Patcher(df, 3, ws.max_rows)
ws.update_batch(patch.patch)

### 5 вкладка

5. Регулярка — статистика - отчет по автоматическим рассылкам строится на основе файла отчета из MindBox, с помощью выгрузки: Отчёты -> Сводная статистика по рассылкам -> Выбор дат -> Экспорт CSV файла

In [ ]:
# MindBox report
df = r.get_report('report.xlsx', 0)
df = df[df.channel == 'Ручные рассылки']

In [ ]:
ws = conn.get_sheet_by_name("5. Регулярка — статистика")
ws = g.gCanvas(ws)
patch = s.Patcher(df, 4, ws.max_rows)
ws.update_batch(patch.patch)

### 8 вкладка

Стратегическая сегментация по покупкам строится на основе файла регистраций клиентов (data/cohort/clients.csv),
его необходимо обновлять при запуске с помощью выгрузки из Mindbox: Клиент -> Действие -> Первое в истории клиента до регистрации. Поля для экспорта email, внешний идентификатор, дата регистрации.

In [17]:
b = imp.load_source('segmentation', 'modules/tab8.py')

In [18]:
orders = pd.read_feather('data/cohort/orders.f')

In [19]:
dates = []
max_date = orders.date_ord.max()
for i in range(15):
    dates.append(max_date)
    max_date -= timedelta(days=7)

In [20]:
Новые подписчики
Старые подписчики
Новые клиенты
Активные (частые)
Активные (остальные)
Предотток (активные)
Предотток (новые)
Отток (активные)
Отток (новые)

segments = [b.build_segmentation(orders, date) for date in dates]
pd.DataFrame(segments).sort_index(ascending=False).T

,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0
flws_new,7169,7603,7202,6882,6984,6974,8064,11064,12943,10646,8747,9403,12067,11098,8533
flws_old,299902,303475,306995,310931,314434,317832,321333,324752,329184,335355,341975,346452,350899,355702,362298
cli_new,24158,25323,23621,25095,25865,27591,28063,28964,33997,32264,32200,29687,38521,17865,29399
prechurn_new,53129,54890,57105,58716,60455,61635,63468,64983,66579,71407,74393,77164,79408,87060,84000
prechurn,645,632,677,754,735,737,753,713,692,739,780,845,967,1144,1924
churn_new,85328,87993,90836,93653,96741,100339,103630,107297,111241,114419,117968,121645,125593,129859,135050
churn,29914,31088,32254,33496,34694,35976,37251,38615,40071,41421,42858,44376,45933,47442,49211
active,164,196,175,166,163,180,195,209,341,317,619,191,235,172,197
active_rest,1753,1865,1887,1878,1866,1860,1865,2038,2101,2308,2536,4599,4252,4135,1805


In [ ]:
#сохранить можно так
pd.DataFrame(segments).sort_index(ascending=False).T.to_csv('segmentation.csv')